# Model-Serving

In [1]:
from snowflake.snowpark import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions

conn = SnowflakeLoginOptions("sanju")
session = Session.builder.configs(conn).create()
session.use_warehouse("ADHOC_WH")
session.use_database("DATA_ALCHEMIST")
session.use_schema("DIAMONDS")

SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


In [3]:
df = session.table("diamonds_transform")
_, test_df = df.random_split(weights=[0.9, 0.1], seed=0)
df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUT_OHE_FAIR"  |"CUT_OHE_GOOD"  |"CUT_OHE_IDEAL"  |"CUT_OHE_PREMIUM"  |"CUT_OHE_VERY_GOOD"  |"COLOR_OHE_D"  |"COLOR_OHE_E"  |"COLOR_OHE_F"  |"COLOR_OHE_G"  |"COLOR_OHE_H"  |"COLOR_OHE_I"  |"COLOR_OHE_J"  |"CLARITY_OHE_I1"  |"CLARITY_OHE_IF"  |"CLARITY_OHE_SI1"  |"CLARITY_OHE_SI2"  |"CLARITY_OHE_VS1"  |"CLARITY_OHE_VS2"  |"CLARITY_OHE_VVS1"  |"CLARITY_OHE_VVS2"  |"CUT_OE"  |"CLARITY_OE"  |"COLOR_OE"  |"COLOR"  |"CLARITY"  |"PRICE"  |"CUT"  |"CARAT"  |"X"   |"Y"   |"Z"   |"DEPTH

List all models in current registry

In [4]:
from snowflake.ml.registry import Registry

registry = Registry(session=session)
print("Models:"); registry.show_models()

Models:


,created_on,name,model_type,database_name,schema_name,comment,owner,default_version_name,versions,aliases
0,2025-05-04 19:55:44.027000-07:00,DIAMOND_PRICE_PREDICTOR,USER_MODEL,DATA_ALCHEMIST,DIAMONDS,None,ACCOUNTADMIN,V1,"[""V1""]","{""DEFAULT"":""V1"",""FIRST"":""V1"",""LAST"":""V1""}"


Describe our model

In [5]:
model = registry.get_model("DIAMOND_PRICE_PREDICTOR")
mv = model.default
print("Functions:"); mv.show_functions()

Functions:


[{'name': 'EXPLAIN',
  'target_method': 'explain',
  'target_method_function_type': 'TABLE_FUNCTION',
  'signature': ModelSignature(
                      inputs=[
                          FeatureSpec(dtype=DataType.DOUBLE, name='CUT_OE', nullable=True),
  		FeatureSpec(dtype=DataType.DOUBLE, name='COLOR_OE', nullable=True),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CLARITY_OE', nullable=True),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CARAT', nullable=True),
  		FeatureSpec(dtype=DataType.DOUBLE, name='DEPTH', nullable=True),
  		FeatureSpec(dtype=DataType.DOUBLE, name='TABLE_PCT', nullable=True),
  		FeatureSpec(dtype=DataType.DOUBLE, name='X', nullable=True),
  		FeatureSpec(dtype=DataType.DOUBLE, name='Y', nullable=True),
  		FeatureSpec(dtype=DataType.DOUBLE, name='Z', nullable=True)
                      ],
                      outputs=[
                          FeatureSpec(dtype=DataType.DOUBLE, name='"CUT_OE_explanation"', nullable=True),
  		FeatureSpec(dtype=DataType

Run predictions

In [ ]:
pred = mv.run(test_df,function_name='predict')      
pred.select("PRICE", "PREDICTED_PRICE").show()

-------------------------------
|"PRICE"  |"PREDICTED_PRICE"  |
-------------------------------
|1960     |2049.34            |
|1107     |1035.33            |
|506      |477.57             |
|394      |400.1              |
|17841    |16864.77           |
|3478     |3565.01            |
|3539     |3424.09            |
|2759     |2414.76            |
|2893     |2750.83            |
|2056     |2080.0             |
-------------------------------



Metrics:

In [8]:
from snowflake.ml.modeling.metrics import mean_absolute_percentage_error

mape = mean_absolute_percentage_error(
    df=pred, 
    y_true_col_names="PRICE", 
    y_pred_col_names="PREDICTED_PRICE")
print(f"MAPE: {mape}")

MAPE: 0.0638390655184737
